In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
BSinfo = pd.read_csv("/content/Base Station basic information.csv")
CLdata = pd.read_csv("/content/Cell-level data.csv")
ECdata = pd.read_csv("/content/Energy consumption data.csv")
power =  pd.read_csv("/content/power_consumption_prediction.csv")

In [ ]:
merged_df = pd.merge(CLdata, ECdata, on=['Time', 'BS'])

In [ ]:
train = pd.merge(BSinfo, merged_df, on='BS')

In [ ]:
train["Time"]=pd.to_datetime(train["Time"])
power["Time"]=pd.to_datetime(power["Time"])
def datetime_features(df_temp):
    """
    Datetime feature üretir.
    """
    df_temp['month'] = df_temp['Time'].dt.month
    df_temp['day'] = df_temp['Time'].dt.day
    df_temp['year'] = df_temp['Time'].dt.year
    df_temp['dayofweek'] = df_temp['Time'].dt.dayofweek
    df_temp['quarter'] = df_temp['Time'].dt.quarter
    df_temp['dayofmonth'] = df_temp['Time'].dt.day
    df_temp['weekofyear'] = df_temp['Time'].dt.weekofyear
    df_temp['is_month_start'] = df_temp.Time.dt.is_month_start.astype(np.int8)
    df_temp['is_month_end'] = df_temp.Time.dt.is_month_end.astype(np.int8)
    df_temp['monday'] = df_temp.Time.dt.weekday.eq(0).astype(np.uint8)
    df_temp['tuesday'] = df_temp.Time.dt.weekday.eq(1).astype(np.uint8)
    df_temp['wednesday'] = df_temp.Time.dt.weekday.eq(2).astype(np.uint8)
    df_temp['thursday'] = df_temp.Time.dt.weekday.eq(3).astype(np.uint8)
    df_temp['friday'] = df_temp.Time.dt.weekday.eq(4).astype(np.uint8)
    df_temp['saturday'] = df_temp.Time.dt.weekday.eq(5).astype(np.uint8)
    df_temp['sunday'] = df_temp.Time.dt.weekday.eq(6).astype(np.uint8)
    df_temp['is_quarter_end'] = df_temp['Time'].dt.is_quarter_end
    df_temp['is_quarter_start'] = df_temp['Time'].dt.is_quarter_start
    df_temp['is_year_end'] = df_temp['Time'].dt.is_year_end
    df_temp['is_year_start'] = df_temp['Time'].dt.is_year_start
    return df_temp
train=datetime_features(train)
test=datetime_features(power)

<ipython-input-132-ddb9f5bc8e0d>:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_temp['weekofyear'] = df_temp['Time'].dt.weekofyear
<ipython-input-132-ddb9f5bc8e0d>:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_temp['weekofyear'] = df_temp['Time'].dt.weekofyear


In [ ]:
train.head(2)

,BS,CellName_x,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower,Time,CellName_y,...,tuesday,wednesday,thursday,friday,saturday,sunday,is_quarter_end,is_quarter_start,is_year_end,is_year_start
0,B_0,Cell0,Type1,Mode2,365.0,20,4,6.875934,2023-01-01 01:00:00,Cell0,...,0,0,0,0,0,1,False,True,False,True
1,B_0,Cell0,Type1,Mode2,365.0,20,4,6.875934,2023-01-01 02:00:00,Cell0,...,0,0,0,0,0,1,False,True,False,True


In [ ]:
target = train['Energy']
train = train.drop(['Time','Energy'],1)
test = test.drop(['Time','Energy'],1)

<ipython-input-134-d6147ea1fc44>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train = train.drop(['Time','Energy'],1)
<ipython-input-134-d6147ea1fc44>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test = test.drop(['Time','Energy'],1)


In [ ]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
common_columns = list(set(train.columns).intersection(test.columns))
train = train[common_columns]
test = test[common_columns]

In [ ]:
train = train.values.astype(np.float32)
target = target.values.astype(np.float32)

In [ ]:
test = test.values.astype(np.float32)

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

In [ ]:
encoding_dim = 300
input_layer = Input(shape=(train.shape[1],))


encoded = Dense(226, activation='relu')(input_layer)
encoded = Dense(encoding_dim, activation='relu')(encoded)

decoded = Dense(226, activation='relu')(encoded)
decoded = Dense(train.shape[1], activation='linear')(decoded)

autoencoder = Model(input_layer, decoded)


autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(train, target, epochs=10, batch_size=16, shuffle=True)

encoder = Model(input_layer, encoded)
train = encoder.predict(train)
test = encoder.predict(test)

Epoch 1/100
7856/7856 [==============================] - 83s 10ms/step - loss: 183.6271
Epoch 2/100
7856/7856 [==============================] - 83s 11ms/step - loss: 49.7610
Epoch 3/100
7856/7856 [==============================] - 82s 10ms/step - loss: 44.8509
Epoch 4/100
7856/7856 [==============================] - 85s 11ms/step - loss: 43.4355
Epoch 5/100
7856/7856 [==============================] - 82s 10ms/step - loss: 42.5861
Epoch 6/100
7856/7856 [==============================] - 81s 10ms/step - loss: 42.3369
Epoch 7/100
7856/7856 [==============================] - 86s 11ms/step - loss: 41.0914
Epoch 8/100
7856/7856 [==============================] - 81s 10ms/step - loss: 40.6495
Epoch 9/100
7856/7856 [==============================] - 80s 10ms/step - loss: 40.6286
Epoch 10/100
7856/7856 [==============================] - 81s 10ms/step - loss: 40.3295
Epoch 11/100
7856/7856 [==============================] - 80s 10ms/step - loss: 39.6002
Epoch 12/100
7856/7856 [================

In [ ]:
power.head(2)

,Time,BS,Energy,w
0,2023-01-01 06:00:00,B_0,NaN,1
1,2023-01-01 11:00:00,B_0,NaN,1


In [ ]:
BSinfo.head(2)

,BS,CellName,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,B_0,Cell0,Type1,Mode2,365.0,20,4,6.875934
1,B_1,Cell0,Type2,Mode2,532.0,20,4,6.875934


In [ ]:
CLdata.head(2)

,Time,BS,CellName,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,ESMode6
0,1/1/2023 1:00,B_0,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0
1,1/1/2023 2:00,B_0,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0


In [ ]:
ECdata.head(2)

,Time,BS,Energy
0,1/1/2023 1:00,B_0,64.275037
1,1/1/2023 2:00,B_0,55.904335


In [ ]:
missing_values = BSinfo.isnull().sum()
print("Missing values in BSinfo:", missing_values)
missing_values = CLdata.isnull().sum()
print("Missing values in CLdata:", missing_values)
missing_values = ECdata.isnull().sum()
print("Missing values in ECdata:", missing_values)

Missing values in BSinfo: BS           0
CellName     0
RUType       0
Mode         0
Frequency    0
Bandwidth    0
Antennas     0
TXpower      0
dtype: int64
Missing values in CLdata: Time        0
BS          0
CellName    0
load        0
ESMode1     0
ESMode2     0
ESMode3     0
ESMode4     0
ESMode5     0
ESMode6     0
dtype: int64
Missing values in ECdata: Time      0
BS        0
Energy    0
dtype: int64


In [ ]:
# Check for invalid data types
invalid_data_types = []
for column in BSinfo.columns:
    if BSinfo[column].dtype.name == "object":
        invalid_data_types.append(column)
print("Invalid data types in BSinfo:", invalid_data_types)


Invalid data types in BSinfo: ['BS', 'CellName', 'RUType', 'Mode']


In [ ]:
test = power
train = ECdata

target = train['Energy']
train = train.drop('Energy',1)
test = test.drop(['Energy','w'],1)

<ipython-input-9-3638b1961baa>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train = train.drop('Energy',1)
<ipython-input-9-3638b1961baa>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test = test.drop(['Energy','w'],1)


In [ ]:
train.head()

,Time,BS
0,1/1/2023 1:00,B_0
1,1/1/2023 2:00,B_0
2,1/1/2023 3:00,B_0
3,1/1/2023 4:00,B_0
4,1/1/2023 5:00,B_0


In [ ]:
test.head()

,Time,BS
0,2023-01-01 06:00:00,B_0
1,2023-01-01 11:00:00,B_0
2,2023-01-01 12:00:00,B_0
3,2023-01-01 13:00:00,B_0
4,2023-01-01 23:00:00,B_0


In [ ]:
train["Time"]=pd.to_datetime(train["Time"])
test["Time"]=pd.to_datetime(test["Time"])
def datetime_features(df_temp):
    """
    Datetime feature üretir.
    """
    df_temp['month'] = df_temp['Time'].dt.month
    df_temp['day'] = df_temp['Time'].dt.day
    df_temp['year'] = df_temp['Time'].dt.year
    df_temp['dayofweek'] = df_temp['Time'].dt.dayofweek
    df_temp['quarter'] = df_temp['Time'].dt.quarter
    df_temp['dayofmonth'] = df_temp['Time'].dt.day
    df_temp['weekofyear'] = df_temp['Time'].dt.weekofyear
    df_temp['is_month_start'] = df_temp.Time.dt.is_month_start.astype(np.int8)
    df_temp['is_month_end'] = df_temp.Time.dt.is_month_end.astype(np.int8)
    df_temp['monday'] = df_temp.Time.dt.weekday.eq(0).astype(np.uint8)
    df_temp['tuesday'] = df_temp.Time.dt.weekday.eq(1).astype(np.uint8)
    df_temp['wednesday'] = df_temp.Time.dt.weekday.eq(2).astype(np.uint8)
    df_temp['thursday'] = df_temp.Time.dt.weekday.eq(3).astype(np.uint8)
    df_temp['friday'] = df_temp.Time.dt.weekday.eq(4).astype(np.uint8)
    df_temp['saturday'] = df_temp.Time.dt.weekday.eq(5).astype(np.uint8)
    df_temp['sunday'] = df_temp.Time.dt.weekday.eq(6).astype(np.uint8)
    df_temp['is_quarter_end'] = df_temp['Time'].dt.is_quarter_end
    df_temp['is_quarter_start'] = df_temp['Time'].dt.is_quarter_start
    df_temp['is_year_end'] = df_temp['Time'].dt.is_year_end
    df_temp['is_year_start'] = df_temp['Time'].dt.is_year_start
    return df_temp
train=datetime_features(train)
test=datetime_features(test)

<ipython-input-12-63882d4a7cef>:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_temp['weekofyear'] = df_temp['Time'].dt.weekofyear
<ipython-input-12-63882d4a7cef>:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_temp['weekofyear'] = df_temp['Time'].dt.weekofyear


In [ ]:
traincolumns = ['month', 'day', 'dayofweek', 'quarter', 'dayofmonth',
       'weekofyear', 'is_month_start']

train = train[traincolumns]
test = test[traincolumns]

In [ ]:
train.head(2)

,month,day,dayofweek,quarter,dayofmonth,weekofyear,is_month_start
0,1,1,6,1,1,52,1
1,1,1,6,1,1,52,1


In [ ]:
test.head(2)

,month,day,dayofweek,quarter,dayofmonth,weekofyear,is_month_start
0,1,1,6,1,1,52,1
1,1,1,6,1,1,52,1


In [ ]:
train = train.drop(['Time','BS'],1)
test = test.drop(['Time','BS'],1)

<ipython-input-147-28d4439d1e3d>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train = train.drop(['Time','BS'],1)


KeyError: ignored

In [ ]:
train.shape

(92629, 16)

In [ ]:
test.shape

(26139, 16)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.1, random_state=43)


In [ ]:
import xgboost as xgb
import optuna

def objective(trial):
    param = {
        'objective': 'reg:squarederror',
        'booster': 'gbtree',
        'eval_metric': 'rmse',
        'verbosity': 0,
        'tree_method': 'auto',
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-8, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-8, 1.0),
        'max_delta_step': trial.suggest_loguniform('max_delta_step', 1e-8, 1.0),
        'scale_pos_weight': trial.suggest_loguniform('scale_pos_weight', 1e-8, 1.0),
    }

    dtrain = xgb.DMatrix(x_train, label=y_train)
    dval = xgb.DMatrix(x_test, label=y_test)

    evals_result = {}
    model = xgb.train(param, dtrain, num_boost_round=100,
                      evals=[(dtrain, 'train')], evals_result=evals_result,
                      early_stopping_rounds=100, verbose_eval=False)

    return evals_result['train']['rmse'][-1]

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2023-07-31 03:48:51,621] A new study created in memory with name: no-name-189910fe-fb22-4ab5-939b-a581c09e2408
<ipython-input-144-dee8c7a6416f>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
<ipython-input-144-dee8c7a6416f>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
<ipython-input-144-dee8c7a6416f>:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform(

Best trial:
  Value:  16.033216715220053
  Params: 
    lambda: 0.022614273193020028
    alpha: 1.0913525913795983e-07
    learning_rate: 0.2680224177499454
    n_estimators: 740
    max_depth: 4
    min_child_weight: 7.828475595193602e-06
    subsample: 0.9653283779648033
    colsample_bytree: 0.3817313207457798
    colsample_bylevel: 0.13524653963778868
    colsample_bynode: 0.6358743650823659
    gamma: 0.026213387764757838
    max_delta_step: 0.9915483339979752
    scale_pos_weight: 0.003594952698651601


In [ ]:
best_params = {
    'objective': 'reg:squarederror',
    'booster': 'gbtree',
    'eval_metric': 'rmse',
    'verbosity': 0,
    'tree_method': 'auto',
    'lambda': 0.022614273193020028,
    'alpha': 1.0913525913795983e-07,
    'learning_rate': 0.2680224177499454,
    'n_estimators': 740,
    'max_depth': 4,
    'min_child_weight': 7.828475595193602e-06,
    'subsample': 0.9653283779648033,
    'colsample_bytree': 0.3817313207457798,
    'colsample_bylevel': 0.13524653963778868,
    'colsample_bynode': 0.6358743650823659,
    'gamma': 0.026213387764757838,
    'max_delta_step': 0.9915483339979752,
    'scale_pos_weight': 0.003594952698651601,
}

dtrain = xgb.DMatrix(x_train, label=y_train)
dval = xgb.DMatrix(x_test, label=y_test)

evals_result = {}
model = xgb.train(params=best_params, dtrain=dtrain, num_boost_round=100,
                  evals=[(dtrain, 'train'), (dval, 'eval')],
                  evals_result=evals_result, early_stopping_rounds=100, verbose_eval=False)


In [ ]:
test = xgb.DMatrix(test)

In [ ]:
pred = model.predict(test)

In [ ]:
testSup =  pd.read_csv("/content/power_consumption_prediction.csv")

In [ ]:
testSup['Time_BS'] = testSup.apply(lambda row: f"{row['Time']}_{row['BS']}", axis=1)

In [ ]:
submission = pd.DataFrame({'ID': testSup["Time_BS"], 'Energy': preds})
submission.to_csv('subb 7bb.csv', index=False)

In [ ]:
submission

,ID,Energy
0,2023-01-01 06:00:00_B_0,73.216241
1,2023-01-01 11:00:00_B_0,73.216241
2,2023-01-01 12:00:00_B_0,73.216241
3,2023-01-01 13:00:00_B_0,73.216241
4,2023-01-01 23:00:00_B_0,73.216241
...,...,...
26134,2023-01-02 19:00:00_B_1019,47.329599
26135,2023-01-02 20:00:00_B_1019,47.329599
26136,2023-01-02 21:00:00_B_1019,47.329599
26137,2023-01-02 22:00:00_B_1019,47.329599
